user_id: 사용자의 고유 ID,
movie_id: 영화의 고유 ID,
rating: 사용자가 영화에 준 평점
timestamp: 평점이 기록된 시간

In [46]:
# 별점 1,2,3, 별로 카운트

In [47]:
from pyspark import SparkConf, SparkContext
# 스파크 환경 설정 객체 생성
conf = SparkConf().setMaster("local").setAppName("241204_02_MovieLens")
spark = SparkContext(conf=conf).getOrCreate()
spark

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=241204_02_MovieLens, master=local) created by __init__ at /tmp/ipykernel_4392/798159502.py:4 

In [48]:
import os
directory = os.path.join(os.getcwd(), "data")
filename = "u.data"
filepath = os.path.join(directory, filename)

lines = spark.textFile("file:///"+filepath.replace("\\", "/"))
lines.take(5)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596']

In [112]:
#1번째 방법
movie_infor = lines.map(lambda x: x.split('\t'))
movie_infor.take(5)

[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['22', '377', '1', '878887116'],
 ['244', '51', '2', '880606923'],
 ['166', '346', '1', '886397596']]

In [111]:
#2번째 방법
def parse(row):
    fields = row.split("\t")
    userid = fields[0]
    movie = fields[1]
    rate = fields[2]
    
    return movie,userid,rate

In [109]:
movie_info = lines.map(parse)
movie_info.take(10)

[('242', '196', '3'),
 ('302', '186', '3'),
 ('377', '22', '1'),
 ('51', '244', '2'),
 ('346', '166', '1'),
 ('474', '298', '4'),
 ('265', '115', '2'),
 ('465', '253', '5'),
 ('451', '305', '3'),
 ('86', '6', '3')]

## 총 유저 수,영화 수,평점 별 카운트

In [57]:
total_user = movie_info.map(lambda x: x[0]).distinct()
total_user.count()

1682

In [58]:
total_movie = movie_info.map(lambda x: x[1]).distinct()
total_movie.count()

943

In [105]:
#sort_rate_sum = lines.map(lambda x: x.split('\t')[2])
#sort_rate_sum.countByValue()

defaultdict(int, {'3': 27145, '1': 6110, '2': 11370, '4': 34174, '5': 21201})

In [110]:
sort_rate_sum = movie_info.map(lambda x: x[2])
sort_rate_sum.countByValue()

defaultdict(int, {'3': 27145, '1': 6110, '2': 11370, '4': 34174, '5': 21201})

## 유저 별 평점,수,평균

In [100]:
user_rate = movie_info.map(lambda x: (x[0], x[2]))
user_rate.take(5)

[('242', '3'), ('302', '3'), ('377', '1'), ('51', '2'), ('346', '1')]

In [101]:
user_rate_cnt = user_rate.mapValues(lambda x: (x,1))
user_rate_cnt.take(5)

[('242', ('3', 1)),
 ('302', ('3', 1)),
 ('377', ('1', 1)),
 ('51', ('2', 1)),
 ('346', ('1', 1))]

In [102]:
user_rating = user_rate_cnt.reduceByKey(lambda x, y: (str(int(x[0]) + int(y[0])), x[1] + y[1]))
user_rating.take(5)

[('242', ('467', 117)),
 ('302', ('1236', 297)),
 ('377', ('28', 13)),
 ('51', ('280', 81)),
 ('346', ('459', 126))]

In [103]:
user_rating_avg = user_rating.mapValues(lambda x: int(x[0])/x[1])
user_rating_avg.collect()

[('242', 3.9914529914529915),
 ('302', 4.161616161616162),
 ('377', 2.1538461538461537),
 ('51', 3.45679012345679),
 ('346', 3.642857142857143),
 ('474', 4.252577319587629),
 ('265', 3.8634361233480177),
 ('465', 3.5647058823529414),
 ('451', 3.347058823529412),
 ('86', 3.94),
 ('257', 3.745874587458746),
 ('1014', 3.061224489795918),
 ('222', 3.66027397260274),
 ('40', 2.8947368421052633),
 ('29', 2.6666666666666665),
 ('785', 3.1538461538461537),
 ('387', 3.3846153846153846),
 ('274', 3.5),
 ('1042', 3.142857142857143),
 ('1184', 2.5),
 ('392', 3.5441176470588234),
 ('486', 3.796875),
 ('144', 3.8724279835390947),
 ('118', 3.2150170648464163),
 ('1', 3.8783185840707963),
 ('546', 3.031496062992126),
 ('95', 3.8127853881278537),
 ('768', 3.076923076923077),
 ('277', 3.464788732394366),
 ('234', 3.775),
 ('246', 3.935483870967742),
 ('98', 4.28974358974359),
 ('193', 3.917197452229299),
 ('88', 3.539906103286385),
 ('194', 4.058091286307054),
 ('1081', 2.75),
 ('603', 4.387559808612440

## 영화 별 평점,평균

In [83]:
movie_rate = movie_info.map(lambda x: (x[1], x[2]))
movie_rate.take(5)


[('196', '3'), ('186', '3'), ('22', '1'), ('244', '2'), ('166', '1')]

In [85]:
movie_rate_cnt = movie_rate.mapValues(lambda x: (x,1))
movie_rate_cnt.take(5)

[('196', ('3', 1)),
 ('186', ('3', 1)),
 ('22', ('1', 1)),
 ('244', ('2', 1)),
 ('166', ('1', 1))]

In [92]:
movie_rating = movie_rate_cnt.reduceByKey(lambda x, y: (str(int(x[0]) + int(y[0])), x[1] + y[1]))
movie_rating.take(5)

[('196', ('141', 39)),
 ('186', ('314', 92)),
 ('22', ('429', 128)),
 ('244', ('869', 238)),
 ('166', ('71', 20))]

In [99]:
movie_rating_avg = movie_rating.mapValues(lambda x: int(x[0])/x[1])
movie_rating_avg.collect()

[('196', 3.6153846153846154),
 ('186', 3.4130434782608696),
 ('22', 3.3515625),
 ('244', 3.6512605042016806),
 ('166', 3.55),
 ('298', 4.031496062992126),
 ('115', 3.9347826086956523),
 ('253', 3.979381443298969),
 ('305', 3.40990990990991),
 ('6', 3.6350710900473935),
 ('62', 3.3017241379310347),
 ('286', 3.7118055555555554),
 ('200', 4.032407407407407),
 ('210', 4.0606060606060606),
 ('224', 2.99290780141844),
 ('303', 3.365702479338843),
 ('122', 3.9836065573770494),
 ('194', 2.963934426229508),
 ('291', 3.685810810810811),
 ('234', 3.122916666666667),
 ('119', 3.9502762430939224),
 ('167', 3.3768115942028984),
 ('299', 3.4642857142857144),
 ('308', 3.7581863979848866),
 ('95', 3.41726618705036),
 ('38', 3.71900826446281),
 ('102', 2.615740740740741),
 ('63', 3.118279569892473),
 ('160', 3.9166666666666665),
 ('50', 3.5416666666666665),
 ('301', 3.5272727272727273),
 ('225', 4.518518518518518),
 ('290', 3.3533333333333335),
 ('97', 4.158730158730159),
 ('157', 3.784313725490196),
 (